In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/dataset/Reviews.csv")

In [ ]:
pip install nltk

In [ ]:
import re
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split





In [ ]:
df = df[['Text', 'Summary']]
df

In [ ]:
print(len(df))
df.drop_duplicates(subset=['Text'],inplace=True) 
print(len(df))

In [ ]:
df.dropna(axis=0, inplace=True)
print(df.isna().sum())

In [ ]:
contraction_mapping = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "here's": "here is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}


In [ ]:
stop_words = set(stopwords.words('english'))

def text_cleaner(text):
    new_str = text.lower()
    new_str = BeautifulSoup(new_str, "lxml").text
    new_str = re.sub(r'\([^)]*\)', '', new_str)
    new_str = re.sub('"', '', new_str)
    new_str = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_str.split(" ")])
    new_str = re.sub(r"'s\b", "", new_str)
    new_str = re.sub("[^a-zA-Z]", " ", new_str) 
    tokens = [w for w in new_str.split() if not w in stop_words]
    long_words = []
    for i in tokens:
        if len(i) >= 3:  
            long_words.append(i)
    return (" ".join(long_words)).strip() 


cleantxt = []
for t in df['Text']:
    cleaned_text.append(text_cleaner(t))

In [ ]:
def summary_cleaner(text):
    new_str = re.sub('"', '', str(text))
    new_str = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_str.split(" ")])  
    new_str = re.sub(r"'s\b", "", new_str)
    new_str = re.sub("[^a-zA-Z]", " ", new_str)
    new_str = new_str.lower()
    tokens = new_str.split()
    new_str = ''
    for i in tokens:
        if len(i) > 1:
            new_str = new_str+i+' '
    return new_str


cleansum = []
for tmmm in df['Summary']:
    cleaned_summary.append(summary_cleaner(tmmm))

In [ ]:
df['cleaned_text'] = cleantxt
df['cleaned_summary'] = cleansum
df['cleaned_summary'].replace('', np.nan, inplace=True)

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df = df[['Text','Summary','cleaned_text','cleaned_summary']]
df

In [ ]:
df.to_csv('yehbhi.csv', index=False)

In [ ]:
import pandas as pd
from IPython.display import FileLink


# Create a download link for the CSV file
FileLink(r'yehbhi.csv')


In [ ]:
sum_all_tokens = sum([len(review.split()) for review in df['cleaned_text']])
avg_length = sum_all_tokens / len(df['cleaned_text'])
avg_length

In [ ]:
sum_all_tokens = sum([len(review.split()) for review in df['cleaned_summary']])
avg_length = sum_all_tokens / len(df['cleaned_summary'])
avg_length

In [ ]:
df=pd.read_csv("/kaggle/input/finale/lpl.csv")
df = df.sample(n=500, random_state=0)

In [ ]:
df=df[['cleaned_text','cleaned_summary']]

In [ ]:
df

In [ ]:
import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)


tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<|startoftext|>",
                              "eos_token": "<|endoftext|>",
                             })

model.resize_token_embeddings(len(tokenizer))


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

class SummaryDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.data = df
        self.texts = self.data['cleaned_text'].values
        self.summaries = self.data['cleaned_summary'].values

        self.X = []

        for text, summary in zip(self.texts, self.summaries):
            self.X.append(text + "  " + summary + " ")

        self.X_encoded = tokenizer(self.X, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token 

train_dataset = SummaryDataset(train_df, tokenizer)
test_dataset = SummaryDataset(test_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
len(train_dataset.__getitem__(0)[0])

In [ ]:
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cpu")
device
model.to(device)

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

# Assuming you have defined your model and tokenizer elsewhere in your code
# Also assuming you have already defined train_dataloader and test_dataloader

device = torch.device("cpu")

optim = Adam(model.parameters(), lr=1e-3)

def train(dataloader, model, optim, device):
    epochs = 1
    losses = []

    model.to(device)  # Move model to CPU device

    for epoch in range(epochs):
        print(f'Training - Epoch: {epoch + 1} started')
        total_loss = 0
        num_batches = 0

        for input_ids, attention_mask in tqdm(dataloader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            optim.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            loss.backward()
            optim.step()

            total_loss += loss.item()
            num_batches += 1
            losses.append(loss.item())

        avg_loss = total_loss / num_batches
        print(f'Training - Epoch: {epoch + 1}, Loss: {avg_loss:.4f}')

        if avg_loss <= 1.5:
            model.save_pretrained('gpt2_024')
            torch.save(model.state_dict(), 'gpt2_024.pth')
            break

    return losses

def evaluate(dataloader, model, device):
    model.eval()
    total_loss = 0
    num_batches = 0

    with torch.no_grad():
        for input_ids, attention_mask in tqdm(dataloader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            total_loss += loss.item()
            num_batches += 1

    avg_loss = total_loss / num_batches
    print(f'Testing - Loss: {avg_loss:.4f}')
    return avg_loss

train_losses = train(train_dataloader, model, optim, device)
test_loss = evaluate(test_dataloader, model, device)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2_024')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2_024')

# Set device
device = torch.device("cpu")  # or "cuda" if you have GPU

# Define a function to generate summaries
def generate_summary(review_text, model, tokenizer, device):
    # Clean the review text
    cleaned_review = text_cleaner(review_text)
    
    # Tokenize the cleaned review text
    inputs = tokenizer(cleaned_review, return_tensors="pt", max_length=1024, truncation=True, padding="max_length").to(device)
    
    # Generate summary using the model
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100, num_beams=4, early_stopping=True)
    
    # Decode the generated summary
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_summary

# Example review text and summary
s=input()
review_text=s
# review_summary = "Amazing product, exceeded expectations."

# Generate summary for the review text using the fine-tuned model
generated_summary = generate_summary(review_text, model, tokenizer, device)

# Print the generated summary
print("Generated Summary:", generated_summary)


In [ ]:
def text_cleaner(text):
    new_str = text.lower()
    new_str = BeautifulSoup(new_str, "lxml").text
    new_str = re.sub(r'\([^)]*\)', '', new_str)
    new_str = re.sub('"', '', new_str)
    new_str = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_str.split(" ")])
    new_str = re.sub(r"'s\b", "", new_str)
    new_str = re.sub("[^a-zA-Z]", " ", new_str) 
    tokens = [w for w in new_str.split() if not w in stop_words]
    long_words = []
    for i in tokens:
        if len(i) >= 3:  
            long_words.append(i)
    return (" ".join(long_words)).strip() 

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


In [ ]:
# Define a function to generate summaries
def generate_summary(review_text, model, tokenizer, device):
    # Clean the review text
    cleaned_review = text_cleaner(review_text)
    
    # Tokenize the cleaned review text
    inputs = tokenizer(cleaned_review, return_tensors="pt", max_length=1024, truncation=True, padding="max_length").to(device)
    
    # Generate summary using the model
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=1024, num_beams=4, early_stopping=True)
    
    # Decode the generated summary
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_summary


# Example review text and summary
s=input()
review_text=s
# review_summary = "Amazing product, exceeded expectations."

# Generate summary for the review text using the fine-tuned model
generated_summary = generate_summary(review_text, model, tokenizer, device)

# Print the generated summary
print("Generated Summary:", generated_summary)

In [ ]:
pip install rouge

In [ ]:
from rouge import Rouge

# Initialize Rouge
rouge = Rouge()
# Compute ROUGE scores
scores = rouge.get_scores(generated_summary, actual_summary)

# Print ROUGE scores
print("ROUGE-1: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(scores[0]['rouge-1']['p'], scores[0]['rouge-1']['r'], scores[0]['rouge-1']['f']))
print("ROUGE-2: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(scores[0]['rouge-2']['p'], scores[0]['rouge-2']['r'], scores[0]['rouge-2']['f']))
print("ROUGE-L: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(scores[0]['rouge-l']['p'], scores[0]['rouge-l']['r'], scores[0]['rouge-l']['f']))


In [ ]:
###pls note my kaggle went out of memory and kaggle notebook restarted therfore lost all the outputs
###I am trying to run on a very small dataset with a single epoch and 500 split data for output showing however during demo i will train it again as done before the genuine out of memory issue.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Step 1: Initialize a GPT-2 tokenizer and model from Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': ''})  # Add padding token
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 2: Load and split the dataset
data = df  # Replace "your_dataset.csv" with the path to your dataset
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Step 3: Implement a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['cleaned_text']
        summary = self.data.iloc[idx]['cleaned_summary']
        inputs = self.tokenizer.encode_plus(text, add_special_tokens=True, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        labels = self.tokenizer.encode_plus(summary, add_special_tokens=True, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': labels['input_ids'].flatten(),
        }

# Step 4: Fine-tune the GPT-2 model on the review dataset to generate summaries
training_args = TrainingArguments(
    output_dir='./output',  # Specify the output directory
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
)

train_dataset = CustomDataset(train_data, tokenizer, max_length=512)
test_dataset = CustomDataset(test_data, tokenizer, max_length=512)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# Define the directory path to save the fine-tuned model
output_directory = "./fine_tuned_gpt2_model"

# Save the fine-tuned model and tokenizer
model.save_pretrained(output_directory)
tokenizer.save_pretrained(output_directory)


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Define the directory path where the fine-tuned model and tokenizer are saved
output_directory = "./fine_tuned_gpt2_model"

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(output_directory)
tokenizer = GPT2Tokenizer.from_pretrained(output_directory)

# Define the device (either cuda:0 for GPU or cpu for CPU)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Move the model to the specified device
model.to(device)

# Step 1: Preprocess the input review text using the tokenizer
input_review_text = "The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound, and feels comfortable to play. However, some users have reported issues with the tuning stability."
inputs = tokenizer(input_review_text, return_tensors="pt")

# Move input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Step 2: Pass the preprocessed input review text to the fine-tuned model to generate the summary
outputs = model.generate(input_ids=inputs['input_ids'], max_length=50, num_beams=4, early_stopping=True)

# Step 3: Decode the generated summary using the tokenizer
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)


In [ ]:
The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound, and feels comfortable to play. However, some users have reported issues with the tuning stability.

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./output")  # Load from the directory where the model was saved during fine-tuning
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define the device (either cuda:0 for GPU or cpu for CPU)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Move the model to the specified device
model.to(device)

# Step 1: Preprocess the input review text using the tokenizer
input_review_text = "The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound, and feels comfortable to play. However, some users have reported issues with the tuning stability."
inputs = tokenizer(input_review_text, return_tensors="pt")

# Move input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Step 2: Pass the preprocessed input review text to the fine-tuned model to generate the summary
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=50, num_beams=4, early_stopping=True)

# Step 3: Decode the generated summary using the tokenizer
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)
